In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune import CLIReporter


In [4]:
def get_datas(csv_path):
    data = pd.read_csv(csv_path)
    
    # 特徴量とターゲット変数の分割,DFのvalueのみ抽出
    X = data.drop('dengue', axis=1).values  
    y = data['dengue'].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    #numpy配列をtensolに変換
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)
    
    #TensorDatasetの作成
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    
    return train_dataset,train_loader,X_test_tensor,y_test_tensor
    

In [5]:
class ClassificationNN(nn.Module):
    def __init__(self, input_size):
        super(ClassificationNN, self).__init__()
        self.fc1 = nn.Linear(input_size,64)  
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)           
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))         
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))      
        return x


In [6]:
def train_classification_nn(config, train_loader, input_size):
    model = ClassificationNN(input_size)
    criterion = nn.BCELoss()  # バイナリクロスエントロピー
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])

    for epoch in range(config["epochs"]):
        model.train()
        running_loss = 0.0
        for X_batch, y_bach in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_bach.unsqueeze(1).float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        tune.report(loss=running_loss / len(train_loader))
    
    return model

In [7]:
def tune_nn(input_size,train_loader):
    train_loader=train_loader
    
    config = {"lr": tune.loguniform(1e-4, 1e-1),"epochs": tune.choice([10, 20, 30])}

    scheduler = ASHAScheduler(metric="loss",
                              mode="min",max_t=30,
                              grace_period=1,
                              reduction_factor=2)

    reporter = CLIReporter(metric_columns=["loss", "training_iteration"])
    
    result = tune.run(tune.with_parameters(train_classification_nn, data_loader=train_loader, input_size=input_size),
                      resources_per_trial={"cpu": 1, "gpu": 0},
                      config=config,
                      num_samples=10,
                      scheduler=scheduler,
                      progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final loss: {}".format(best_trial.last_result["loss"]))
    
    return best_trial.config

In [8]:
# テストデータでの評価
def test_classification_nn(X_test_tensor,y_test_tensor,model):
    model.eval()  # 評価モード
    with torch.no_grad():
        predictions = model(X_test_tensor)
        predictions = predictions.round()  # 出力を0または1に丸める
        accuracy = (predictions == y_test_tensor).float().mean()
        print(f'Accuracy: {accuracy:.4f}')
        
    return